<a href="https://colab.research.google.com/github/lopamudrapathak/Vlan.py/blob/main/3langidentify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/Dataset.zip -d Dataset

Archive:  /content/Dataset.zip
   creating: Dataset/Dataset/
  inflating: Dataset/Dataset/1.txt   
  inflating: Dataset/Dataset/10.txt  
  inflating: Dataset/Dataset/11.txt  
  inflating: Dataset/Dataset/12.txt  
  inflating: Dataset/Dataset/13.txt  
  inflating: Dataset/Dataset/14.txt  
  inflating: Dataset/Dataset/15.txt  
  inflating: Dataset/Dataset/16.txt  
  inflating: Dataset/Dataset/17.txt  
  inflating: Dataset/Dataset/2.txt   
  inflating: Dataset/Dataset/3.txt   
  inflating: Dataset/Dataset/4.txt   
  inflating: Dataset/Dataset/5.txt   
  inflating: Dataset/Dataset/6.txt   
  inflating: Dataset/Dataset/7.txt   
  inflating: Dataset/Dataset/8.txt   
  inflating: Dataset/Dataset/9.txt   


In [ ]:
old_names_list = [i for i in range(1, 20)]
old_to_new = {
              'Bhojpuri' : [i for i in range(1,6)], # Files with serial no 1 to 10 are in french, 10 to 20 in English and so on
              'Hindi' : [ i for i in range(6,12)],
              'Sanskrit' : [i for i in range(12,17)]
              }

In [ ]:
for lang in old_to_new.keys():
    outfile = 'Dataset/' + lang + '.txt'
    with open( outfile, 'w') as fname: # destination file i.e., language
        for old_file in old_names_list:
            if old_file in old_to_new[lang]:
                with open('/content/Dataset/Dataset/' + str(old_file) + '.txt') as inp:#opening each match of old_to_new[lang]
                    fname.write(inp.read())

In [ ]:
import re
import  string
import numpy as np
import math

In [ ]:
n = [2,3,4]  # Here we are choosing bigrams,trigrams and quadgrams; change this value to get n-grams with a particular n
k = 50 # Decides how many top n-grams will be used for calculating the distance metric

In [ ]:
def create_n_grams(lang, num, k): #returns top k n-grams according to frequency
    words = re.sub('['+string.punctuation+']', '', lang) #  punctuation removed
    words = words.lower()
    words = re.sub('\s+', ' ', words).strip() # replaces multiple spaces, newline tabs with a single space
    words = words.replace(' ','_')# so that we can visualise spaces easily
    grams = {}
    #print (words)
    for i in range(len(words)-num):
        temp = words[i:i+num]
        if temp in grams:
            grams[temp] += 1
        else:
            grams[temp] = 1
    sum_freq = len(words) - num + 1
    for key in grams.keys():
        red = 1 # reduction factor equal 1 if no '_' is present
        if '_' in key: red = 2
        grams[key] = round(math.log(grams[key] / (red * sum_freq)), 3) #normalizing by dividing by total no of n-grams for that corpus and taking log                                             
    grams = sorted(grams.items(), key= lambda x : x[1], reverse = True) 
    #print (grams)
    final_grams = [] # contains a list of top k n-grams in a given language 
    log_probs = [] # contains logprobs corresponding to each n-gram
    for i in range(len(grams)):
        final_grams.append(grams[i][0])
        log_probs.append(grams[i][1])
    return final_grams, log_probs
       
with open ('Dataset/Hindi.txt', 'r') as fname:
    temp = fname.read()  
    create_n_grams(temp, 3, 20) 

In [ ]:
lang_list = ['Bhojpuri', 'Hindi', 'Sanskrit']
bi_grams = {}
tri_grams = {}
quad_grams = {}
for lang in lang_list:
    with open ('Dataset/'+lang+'.txt', 'r') as fname:
        file = fname.read()
        bi_grams[lang] = create_n_grams(file, n[0], k)
        tri_grams[lang] = create_n_grams(file, n[1], k)
        quad_grams[lang] = create_n_grams(file, n[2], k)       
n_grams = {2 : bi_grams, 3 : tri_grams, 4 : quad_grams}
#print (n_grams[2]) 

In [ ]:
def matching_score_2(test_grams, grams_list, n): # n helps us know whether it is bigram, trigram or quadgram
    dist = {lang: 0 for lang in lang_list} # distance corresponding to each language
    for gram in test_grams[0]:
        for lang in grams_list.keys():
            idx_2 = test_grams[0].index(gram)
            if gram in n_grams[n][lang][0] : 
                idx = n_grams[n][lang][0].index(gram)
                dist[lang] += abs(n_grams[n][lang][1][idx] - test_grams[1][idx_2])
            else: # gram is not present in that language's corpus
                dist[lang] += abs(test_grams[1][idx_2])
                # penalty term
    return dist 

In [ ]:
def language_identify_2(file_address, st): # argument 'st' denotes whether you are uploading a file or directly copying text
    test_bigrams = []
    test_trigrams = []
    test_quadgrams = []
    test_file = []
    if st == 'file': # If you are copying a file address
        temp = file_address
        with open(temp, 'r', errors = 'ignore') as fname: # some characters throw an error with 'utf-8'
            file_address = fname.read()
    #print (file_address) 
    test_bigrams = create_n_grams(file_address, 2, k)
    test_trigrams = create_n_grams(file_address, 3, k)
    test_quadgrams = create_n_grams(file_address, 4, k)
    bi_dist = matching_score_2(test_bigrams, bi_grams, 2) 
    tri_dist = matching_score_2(test_trigrams, tri_grams, 3)
    quad_dist = matching_score_2(test_quadgrams, quad_grams, 4) 
    #print (bi_dist, tri_dist)
    final_dist = {}
    for lang in bi_dist.keys():
        final_dist[lang] =bi_dist[lang] + tri_dist[lang] + quad_dist[lang]
    sum_dist = 0
    for dist in final_dist.values():
        sum_dist += dist
    for lang in final_dist.keys():
        final_dist[lang] /= sum_dist
    dist_list = sorted(final_dist.items(), key= lambda x:x[1])     
    #print (dist_list)    
    print ('Predicted language :' + dist_list[0][0] + '\n')
    return dist_list[0][0]

In [ ]:
file_1 = 'Dataset/lang_test.txt'# CHANGE THIS URL TO INCLUDE YOUR OWN TEST FILE
language_identify_2(file_1, 'file')

Predicted language :Bhojpuri



'Bhojpuri'

In [ ]:
file_1 = 'Dataset/lang_test2.txt'# CHANGE THIS URL TO INCLUDE YOUR OWN TEST FILE
language_identify_2(file_1, 'file')

Predicted language :Hindi



'Hindi'

In [ ]:
file_1 = 'Dataset/lang_test3.txt'# CHANGE THIS URL TO INCLUDE YOUR OWN TEST FILE
language_identify_2(file_1, 'file')

Predicted language :Hindi



'Hindi'

In [ ]:
file_1 = 'Dataset/lang_test4.txt'# CHANGE THIS URL TO INCLUDE YOUR OWN TEST FILE
language_identify_2(file_1, 'file')

Predicted language :Bhojpuri



'Bhojpuri'

In [39]:
while (True):
    test_input = input("\n Enter few lines of text: ")
    language_identify_2(test_input, 'not_file')
    user = input("Do you want to test more? (y/Y or N/n)")
    if user == 'N' or user == 'n':
        break


 Enter few lines of text: gdysysy
Predicted language :Bhojpuri

Do you want to test more? (y/Y or N/n)n
